## Introduction

The purpose of this notebook is to prepare and clean data on standardized test scores as well as school information and socioeconomic characteristics. This cleaned dataset will then be ready to be exported for analysis.

## Import libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_theme(style='whitegrid')
import missingno as msno

# Train-test splits
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, KFold, RandomizedSearchCV

# Model preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn import preprocessing
from sklearn.preprocessing import PolynomialFeatures

# Imputation
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import SimpleImputer, IterativeImputer, KNNImputer

# Models
from sklearn import linear_model

# Model metrics and analysis
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error

## The data

Two datasets are cleaned and merged in this notebook. 

The 2016 dataset containing SAT/ACT scores and socioeconomic characteristics for school districts comes from EdGap.org: https://www.edgap.org/#4/37.89/-97.00

The second dataset, providing basic information about each school, was pulled from the National Center for Education Statistics: https://nces.ed.gov/ccd/pubschuniv.asp

### EdGap data (Standardized test scores and socioeconomic characteristics)

For this dataset, the socioeconomic data was gathered from the Census Bureau's American Community Survey. EdGap also reports that the standardized test score data comes from states' departments of education or other public education data releases. While the reliability of the public data releases is undetermined, data coming from the census and the departments of education are considered trustworthy.

EdGap has not indicated any processing of this data. The data was assembled by EdGap however, but since the data is public, if there are any errors we ourselves also can attain access to the original data.

### NCES data (Basic school info)

The school information data is from the [National Center for Education Statistics](https://nces.ed.gov/ccd/pubschuniv.asp). This data set consists of basic identifying information about schools and can be assumed to be of reasonably high quality.


## Load the data

Load the EdGap
 data set

In [2]:
edgap = pd.read_excel('https://raw.githubusercontent.com/brian-fischer/DATA-3320/main/education/EdGap_data.xlsx')

/usr/local/lib/python3.9/dist-packages/openpyxl/worksheet/_reader.py:312: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


Load the school information data

In [3]:
!wget https://www.dropbox.com/s/lkl5nvcdmwyoban/ccd_sch_029_1617_w_1a_11212017.csv?dl=0

--2023-04-25 04:03:40--  https://www.dropbox.com/s/lkl5nvcdmwyoban/ccd_sch_029_1617_w_1a_11212017.csv?dl=0
Resolving www.dropbox.com (www.dropbox.com)... 162.125.3.18, 2620:100:6018:18::a27d:312
Connecting to www.dropbox.com (www.dropbox.com)|162.125.3.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: /s/raw/lkl5nvcdmwyoban/ccd_sch_029_1617_w_1a_11212017.csv [following]
--2023-04-25 04:03:41--  https://www.dropbox.com/s/raw/lkl5nvcdmwyoban/ccd_sch_029_1617_w_1a_11212017.csv
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc57dd0282269c88c66c5210b926.dl.dropboxusercontent.com/cd/0/inline/B61LAna6aQInXj8Mi7m6iQ6cdidAVvQH7INImx0ZQyJha5ODcCQH3Z4P_BqG4tfaeeRAySXwnQL1QKSCkJzirmv6iK-CJJyoq74MX12MoS2qfG_z8BTetJAYNHxQXJ7WMvw0DAUrtHWRO_AUQYWAL8P9rDLohanK2OERnC4GFVHOxQ/file# [following]
--2023-04-25 04:03:41--  https://uc57dd0282269c88c66c5210b926.dl.dropboxusercontent.com/cd/0/inline/B61LAna6a

In [4]:
school_info = pd.read_csv('ccd_sch_029_1617_w_1a_11212017.csv?dl=0', encoding='unicode_escape')

<ipython-input-4-da1a7837c049>:1: DtypeWarning: Columns (6,9,14,15,18,19,21,22,25,26,29,31,35,39,41,42) have mixed types. Specify dtype option on import or set low_memory=False.
  school_info = pd.read_csv('ccd_sch_029_1617_w_1a_11212017.csv?dl=0', encoding='unicode_escape')


## Explore the contents of the data sets

In [5]:
edgap

,NCESSCH School ID,CT Unemployment Rate,CT Pct Adults with College Degree,CT Pct Childre In Married Couple Family,CT Median Household Income,School ACT average (or equivalent if SAT score),School Pct Free and Reduced Lunch
0,100001600143,0.117962,0.445283,0.346495,42820.0,20.433455,0.066901
1,100008000024,0.063984,0.662765,0.767619,89320.0,19.498168,0.112412
2,100008000225,0.056460,0.701864,0.713090,84140.0,19.554335,0.096816
3,100017000029,0.044739,0.692062,0.641283,56500.0,17.737485,0.296960
4,100018000040,0.077014,0.640060,0.834402,54015.0,18.245421,0.262641
...,...,...,...,...,...,...,...
7981,560530200294,0.034549,0.590780,0.720077,64891.0,18.500000,0.232156
7982,560569000311,0.069538,0.633860,0.808841,44603.0,22.300000,0.083871
7983,560569500360,0.035159,0.764566,0.776570,44896.0,21.500000,0.184829
7984,560576200324,0.063877,0.670532,0.879324,92134.0,19.700000,0.112583


In [6]:
school_info

,SCHOOL_YEAR,FIPST,STATENAME,ST,SCH_NAME,LEA_NAME,STATE_AGENCY_NO,UNION,ST_LEAID,LEAID,...,G_10_OFFERED,G_11_OFFERED,G_12_OFFERED,G_13_OFFERED,G_UG_OFFERED,G_AE_OFFERED,GSLO,GSHI,LEVEL,IGOFFERED
0,2016-2017,1,ALABAMA,AL,Sequoyah Sch - Chalkville Campus,Alabama Youth Services,1,NaN,AL-210,100002,...,Yes,Yes,Yes,No,No,No,07,12,High,As reported
1,2016-2017,1,ALABAMA,AL,Camps,Alabama Youth Services,1,NaN,AL-210,100002,...,Yes,Yes,Yes,No,No,No,07,12,High,As reported
2,2016-2017,1,ALABAMA,AL,Det Ctr,Alabama Youth Services,1,NaN,AL-210,100002,...,Yes,Yes,Yes,No,No,No,07,12,High,As reported
3,2016-2017,1,ALABAMA,AL,Wallace Sch - Mt Meigs Campus,Alabama Youth Services,1,NaN,AL-210,100002,...,Yes,Yes,Yes,No,No,No,07,12,High,As reported
4,2016-2017,1,ALABAMA,AL,McNeel Sch - Vacca Campus,Alabama Youth Services,1,NaN,AL-210,100002,...,Yes,Yes,Yes,No,No,No,07,12,High,As reported
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
102178,2016-2017,78,U.S. VIRGIN ISLANDS,VI,GLADYS A. ABRAHAM ELEMENTARY SCHOOL,Saint Thomas - Saint John School District,1,NaN,VI-001,7800030,...,No,No,No,No,No,No,KG,06,Elementary,As reported
102179,2016-2017,78,U.S. VIRGIN ISLANDS,VI,ULLA F MULLER ELEMENTARY SCHOOL,Saint Thomas - Saint John School District,1,NaN,VI-001,7800030,...,No,No,No,No,No,No,KG,06,Elementary,As reported
102180,2016-2017,78,U.S. VIRGIN ISLANDS,VI,YVONNE BOWSKY ELEMENTARY SCHOOL,Saint Thomas - Saint John School District,1,NaN,VI-001,7800030,...,No,No,No,No,No,No,KG,06,Elementary,As reported
102181,2016-2017,78,U.S. VIRGIN ISLANDS,VI,CANCRYN JUNIOR HIGH SCHOOL,Saint Thomas - Saint John School District,1,NaN,VI-001,7800030,...,No,No,No,No,No,No,07,08,Middle,As reported


the school-info dataset has a lot more school encapsulated in the data than the EdGap dataset does.

## Convert data types, if necessary

Both datasets share information regarding the schools, so we will have to identify the school ID as the keys for these datasets. Both datasets have this column labeled as `NCESSCH`. the school_info dataset however, has this as a float variable, we will have to convert it to int64 to match they column in the EdGap dataset.

In [7]:
school_info = school_info[school_info['NCESSCH'].isna() == False]

In [8]:
school_info['NCESSCH'] = school_info['NCESSCH'].astype('int64')

<ipython-input-8-006ea9997674>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  school_info['NCESSCH'] = school_info['NCESSCH'].astype('int64')


Now we can use the NCESSCH, or school ID, as the key to join the dataframes on.

## Are the data suitable for answering the question?

We want to perform quick exploratory data analysis to determine whether the data are sufficient to answer our question. If the data are not sufficient, we do not want to waste time doing anything that will not be productive.

## Select relevant subsets of the data

The school information data set contains a lot of information. We only need the year, school identity, location, and school type information.

Keep the columns `SCHOOL_YEAR`, `NCESSCH`, `MSTATE`, `MZIP`, `SCH_TYPE_TEXT`, `LEVEL`

In [9]:
school_info = school_info[['SCHOOL_YEAR', 'NCESSCH', 'MSTATE', 'MZIP', 'SCH_TYPE_TEXT', 'LEVEL']]

## Rename columns

Rename the columns `NCESSCH School ID`, `CT Pct Adults with College Degree`, `CT Unemployment Rate`, `SCT Pct Childre In Married Couple Family`, `CT Median Household Income`, `School ACT average (or equivalent if SAT score)`, `School Pct Free and Reduced Lunch` to `id`, `percent_college`, `rate_unemployment`, `percent_married`, `median_income`, `average_act`, `percent_lunch`.

In [10]:
edgap = edgap.rename(columns={"NCESSCH School ID":"id", 
              "CT Pct Adults with College Degree":"percent_college",        
              "CT Unemployment Rate":"rate_unemployment", 
              "CT Pct Childre In Married Couple Family":"percent_married",
              "CT Median Household Income":"median_income",
              "School ACT average (or equivalent if SAT score)":"average_act",
              "School Pct Free and Reduced Lunch":"percent_lunch"})

Rename the columns `SCHOOL_YEAR`, `NCESSCH`, `MSTATE`, `MZIP`, `SCH_TYPE_TEXT`, `LEVEL` to `year`, `id`, `state`, `zip_code`, `school_type`, `school_level`

In [11]:
school_info = school_info.rename(columns={'SCHOOL_YEAR':'year', 
                                          'NCESSCH':'id', 
                                          'MSTATE':'state',
                                          'MZIP':'zip_code',
                                          'SCH_TYPE_TEXT':'school_type',
                                          'LEVEL':'school_level'})

## Join data frames 

Merge the dataframes (left) on the `id` column, since those are the school ID's

In [12]:
df_educ = edgap.merge(school_info, how = 'left', on='id')

Let's take a look at the new merged dataframe.

In [13]:
df_educ.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7986 entries, 0 to 7985
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   id                 7986 non-null   int64  
 1   rate_unemployment  7972 non-null   float64
 2   percent_college    7973 non-null   float64
 3   percent_married    7961 non-null   float64
 4   median_income      7966 non-null   float64
 5   average_act        7986 non-null   float64
 6   percent_lunch      7986 non-null   float64
 7   year               7898 non-null   object 
 8   state              7898 non-null   object 
 9   zip_code           7898 non-null   object 
 10  school_type        7898 non-null   object 
 11  school_level       7898 non-null   object 
dtypes: float64(6), int64(1), object(5)
memory usage: 811.1+ KB


## Quality Control

Let's look at a description of all the data in the new merged dataframe.

In [14]:
df_educ.describe(include='all')

,id,rate_unemployment,percent_college,percent_married,median_income,average_act,percent_lunch,year,state,zip_code,school_type,school_level
count,7.986000e+03,7972.000000,7973.000000,7961.000000,7966.000000,7986.000000,7986.000000,7898,7898,7898.0,7898,7898
unique,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,20,6529.0,4,4
top,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2016-2017,TX,10457.0,Regular School,High
freq,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7898,966,11.0,7885,7230
mean,3.321869e+11,0.098730,0.568930,0.633440,52026.905222,20.181532,0.420651,NaN,NaN,NaN,NaN,NaN
std,1.323638e+11,0.058959,0.165704,0.196764,24228.057079,2.595201,0.239754,NaN,NaN,NaN,NaN,NaN
min,1.000016e+11,0.000000,0.091493,0.000000,3589.000000,-3.070818,-0.054545,NaN,NaN,NaN,NaN,NaN
25%,2.105340e+11,0.058655,0.450828,0.523810,36597.250000,18.600000,0.238501,NaN,NaN,NaN,NaN,NaN
50%,3.600085e+11,0.085649,0.554979,0.667594,46833.500000,20.400000,0.381570,NaN,NaN,NaN,NaN,NaN
75%,4.226678e+11,0.123376,0.676571,0.777135,61369.250000,21.910867,0.575447,NaN,NaN,NaN,NaN,NaN


Looka like we have a lot of NaN values. We can check those out later.

Check for out-of-range values or values that do not match what we want to analyze. Either set values to `NaN` or remove the observations, as appropriate.

In [15]:
df_educ.agg(['min', 'max']).round(2)

<ipython-input-15-71805aea5120>:1: FutureWarning: ['year', 'state', 'zip_code', 'school_type', 'school_level'] did not aggregate successfully. If any error is raised this will raise in a future version of pandas. Drop these columns/ops to avoid this warning.
  df_educ.agg(['min', 'max']).round(2)


,id,rate_unemployment,percent_college,percent_married,median_income,average_act,percent_lunch
min,100001600143,0.00,0.09,0.0,3589.0,-3.07,-0.05
max,560583000335,0.59,1.00,1.0,226181.0,32.36,1.00


There is a min of -3.07 for average_act, which in invalid (minimum score is 1). There is also a negative percent_lunch value, which does not make sense.

drop all invalid ACT scores

In [16]:
df_educ = df_educ.drop(df_educ.loc[df_educ['average_act'] <= 0].index)

identify all invalid percent_lunch values

In [17]:
df_educ[df_educ['percent_lunch'] <= 0]

,id,rate_unemployment,percent_college,percent_married,median_income,average_act,percent_lunch,year,state,zip_code,school_type,school_level
325,120144005634,0.102484,0.329146,0.611615,33846.0,20.909646,0.000000,2016-2017,FL,34761,Regular School,High
329,120144007822,0.052551,0.764584,0.758966,77545.0,19.468864,0.000000,2016-2017,FL,32809,Regular School,High
1398,173897003956,0.160519,0.673671,0.554577,68924.0,16.500000,-0.004777,2016-2017,IL,60473,Regular School,High
1422,174098004105,0.040594,0.789430,0.831663,69466.0,21.400000,-0.007792,2016-2017,IL,61571,Regular School,High
1946,210372001486,0.173825,0.901288,0.000000,13480.0,23.000000,0.000000,2016-2017,KY,40475,Regular School,High
...,...,...,...,...,...,...,...,...,...,...,...,...
6787,482325002357,0.022008,0.941176,0.707138,145536.0,26.355311,0.000000,2016-2017,TX,75205,Regular School,High
7655,550426000456,0.045510,0.595078,0.743952,58787.0,21.700000,0.000000,2016-2017,WI,53020,Regular School,High
7723,550750000853,0.037732,0.716785,0.816224,74892.0,24.200000,0.000000,2016-2017,WI,53044,Regular School,High
7936,551662002170,0.066324,0.937226,0.900804,135240.0,25.800000,0.000000,2016-2017,WI,53217,Regular School,High


set all invalid percent_lunch values to NaN

In [18]:
df_educ.loc[df_educ['percent_lunch'] <= 0, 'percent_lunch'] = np.nan

Now let's check what our range of school levels are.

In [19]:
df_educ['school_level'].value_counts()

High            7227
Other            631
Not reported      35
Elementary         2
Name: school_level, dtype: int64

Only high school standardized test scores are relevant, so let's only keep the high school data and drop the other categories.

In [20]:
df_educ = df_educ.loc[df_educ['school_level'] == 'High']

## Identify missing values

Let's look at where all those NaNs fall in the merged dataframe.

In [21]:
df_educ.isna().sum()

id                    0
rate_unemployment    12
percent_college      11
percent_married      20
median_income        16
average_act           0
percent_lunch        67
year                  0
state                 0
zip_code              0
school_type           0
school_level          0
dtype: int64

Now let's put those missing values into context and see what percent each of the missing data takes up, out of the whole dataframe.

In [22]:
df_educ.isna().mean().round(4)*100

id                   0.00
rate_unemployment    0.17
percent_college      0.15
percent_married      0.28
median_income        0.22
average_act          0.00
percent_lunch        0.93
year                 0.00
state                0.00
zip_code             0.00
school_type          0.00
school_level         0.00
dtype: float64

The missing values don't overpower the dataframe. We'll address the missing values after making a train test split.

## Train test split

Split the data into training and testing sets, in order to compare models predicting `average_act`.

Lets include all columns in the X var except for ID (since that's the key) and `average_act` since that is our Y

In [23]:
X = df_educ[df_educ.columns.difference(['id', 'average_act'])]

set `average_act` to Y

In [24]:
Y = df_educ[['average_act']]

Set up the train test split with our new variable designations. We are keeping 20% of the data in the test size.

In [25]:
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=.2)

In [26]:
print(x_train.shape, x_test.shape)

(5781, 10) (1446, 10)


10 variables with the selected data randomized are now included in the train test split.

## Data imputation

Now we deal with replacing missing values.

Iterative imputation used method to replace missing values in the columns corresponding to predictor variables in the analysis.

In [27]:
imputer = IterativeImputer()

Use the imputer to run through the selected variables to create replacement values

In [35]:
imputer.fit(x_train.loc[:,'median_income': 'rate_unemployment'])

IterativeImputer()

replace the x_train with the new replacement values.

In [36]:
x_train.loc[:,'median_income': 'rate_unemployment'] = imputer.transform(x_train.loc[:,'median_income': 'rate_unemployment'])

Check if we replaced all the missing values.

In [37]:
x_train.isna().sum()

median_income        0
percent_college      0
percent_lunch        0
percent_married      0
rate_unemployment    0
school_level         0
school_type          0
state                0
year                 0
zip_code             0
dtype: int64

No more missing values!

Join the training dataframes (x_train and y_train)

In [31]:
df_train = x_train.join(y_train)

Here's the merged training dataframe.

In [32]:
df_train.describe()

,median_income,percent_college,percent_lunch,percent_married,rate_unemployment,average_act
count,5781.000000,5781.000000,5781.000000,5781.000000,5781.000000,5781.000000
mean,52594.370385,0.573342,0.413154,0.640041,0.096703,20.301560
std,24262.782712,0.165311,0.231769,0.192452,0.056709,2.499595
min,2162.738939,0.091493,0.000266,0.000000,0.000000,12.362637
25%,37087.000000,0.455313,0.236207,0.533460,0.058222,18.800000
50%,47240.000000,0.559432,0.374684,0.671889,0.084673,20.500000
75%,62106.000000,0.681247,0.557154,0.780147,0.120930,22.000000
max,226181.000000,1.000000,0.998729,1.000000,0.590278,32.362637


In [33]:
df_train.head()

,median_income,percent_college,percent_lunch,percent_married,rate_unemployment,school_level,school_type,state,year,zip_code,average_act
1377,31177.0,0.445373,0.631848,0.276390,0.190445,High,Regular School,IL,2016-2017,62703,18.000000
4649,50754.0,0.566810,0.347898,0.405907,0.138474,High,Regular School,NC,2016-2017,27529,20.689866
7722,46030.0,0.547182,0.110305,0.662667,0.061224,High,Regular School,WI,2016-2017,54136,22.800000
397,32104.0,0.370311,0.476190,0.547009,0.133428,High,Regular School,FL,2016-2017,33880,17.881563
3282,63377.0,0.626765,0.182174,0.594709,0.034224,High,Regular School,MO,2016-2017,64029,21.800000


The following code was an attempt at making this a tidy format...

In [49]:
df_train_tidy = pd.melt(df_train, var_name = 'characteristic or score', value_vars = ['median_income', 'percent_college', 'percent_lunch', 'percent_married', 'rate_unemployment', 'average_act'])

df_train_tidy

,characteristic or score,value
0,median_income,31177.000000
1,median_income,50754.000000
2,median_income,46030.000000
3,median_income,32104.000000
4,median_income,63377.000000
...,...,...
34681,average_act,21.910867
34682,average_act,18.150183
34683,average_act,22.203907
34684,average_act,20.500000


## Create relevant derived variables as new columns

No derived variables (for now)

## Export the clean .csv files

Export files with the training and testing data

In [85]:
from google.colab import files

df_train.to_csv('clean_education_inequality.csv', encoding = 'utf-8-sig', index=False) 

files.download('clean_education_inequality.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>